<a href="https://colab.research.google.com/github/kbrh3/AI-algorithm-practice/blob/main/StatHW3v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(MASS)  # For LDA and QDA
library(nnet)  # For mlr
library(class) #for knn
library(caret) # For cross-validation

ERROR: Error in library(caret): there is no package called ‘caret’


In [ ]:
library(ISLR)###notice the ISLR package is used instead of ISLR2.

set.seed(1)
train <- sample(392, 196)

ERROR: Error in library(ISLR): there is no package called ‘ISLR’


In [ ]:

##setwd("E:/STAT400_Introduction_to_Statistical_Learning/allRmd/data")
train_data <- read.table("/content/zip.train", header = FALSE)
test_data <- read.table("/content/zip.test", header = FALSE)
dim(train_data)

[1] 7291  257

In [ ]:
# Name the first column as 'Y'
names(train_data)[1] <- "Y" #[1] is the first column, response.
names(test_data)[1] <- "Y"
# Name the subsequent columns as 'pixel1', 'pixel2', ... , 'pixel256'
names(train_data)[-1] <- paste0("pixel", 1:(ncol(train_data)-1))
names(test_data)[-1] <- paste0("pixel", 1:(ncol(test_data)-1))

In [ ]:
classes_five <- c(0, 3, 5, 6, 9)
train_data_five <- train_data[train_data$Y %in% classes_five, ]
test_data_five <- test_data[test_data$Y %in% classes_five, ]

In [ ]:
classes_three <- c(0, 1, 2)
train_data_three <- train_data[train_data$Y %in% classes_three, ]
test_data_three <- test_data[test_data$Y %in% classes_three, ]

In [ ]:
classes_ten <- c(0,1,2,3,4,5,6,7,8,9)
train_data_ten <- train_data[train_data$Y %in% classes_ten, ]
test_data_ten <- test_data[test_data$Y %in% classes_ten, ]

In [ ]:
##question 3

# Fit LDA model
lda_model <- lda(Y ~ ., data = train_data_three)
# Predictions
lda_train_pred <- predict(lda_model, train_data_three)$class
lda_test_pred <- predict(lda_model, test_data_three)$class
# Calculate errors
lda_train_error <- mean(lda_train_pred != train_data_three$Y)
lda_test_error <- mean(lda_test_pred != test_data_three$Y)


ERROR: Error in lda.default(x, grouping, ...): variable 16 appears to be constant within groups


In [ ]:
# Fit logistic regression model
logit_model <- multinom(Y ~ ., data = train_data_three)
# Predictions
logit_train_pred <- predict(logit_model, train_data_three)
logit_test_pred <- predict(logit_model, test_data_three)
# Calculate errors
logit_train_error <- mean(logit_train_pred != train_data_three$Y)
logit_test_error <- mean(logit_test_pred != test_data_three$Y)


# weights:  774 (514 variable)
initial  value 3218.934006 
iter  10 value 30.295097
iter  20 value 9.812887
iter  30 value 2.587927
iter  40 value 1.126169
iter  50 value 0.633505
iter  60 value 0.372125
iter  70 value 0.203700
iter  80 value 0.116252
iter  90 value 0.066790
iter 100 value 0.038870
final  value 0.038870 
stopped after 100 iterations


In [ ]:
cat("Logistic Regression Training Error:", logit_train_error, "\n")
cat("Logistic Regression Test Error:", logit_test_error, "\n")


Logistic Regression Training Error: 0 
Logistic Regression Test Error: 0.0499391 


In [ ]:
# Set of k values to try
k_values <- c(1, 3, 5, 7, 9)

# Loop over k values, apply k-NN, and calculate errors
for (k in k_values) {
    knn_model <- knn(train = train_data_three[ , -which(names(train_data_three) == "Y")],
                     test = test_data_three[ , -which(names(test_data_three) == "Y")],
                     cl = train_data_three$Y, k = k)
    knn_test_pred <- knn_model
    knn_test_error <- mean(knn_test_pred != test_data_three$Y)
    cat(sprintf("k = %d: Test Error = %f\n", k, knn_test_error))
}


k = 1: Test Error = 0.020706
k = 3: Test Error = 0.019488
k = 5: Test Error = 0.018270
k = 7: Test Error = 0.019488
k = 9: Test Error = 0.021924


In [ ]:
install.packages("caret")
library(carat)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘Rcpp’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’




ERROR: Error in library(carat): there is no package called ‘carat’


In [ ]:
##begin question 4
library(MASS) # For lda function
LDA10cv <- function(D) {
# Making sure the outcome variable is in the first column
D <- D[, c("Y", setdiff(names(D), "Y"))]
# Create 10 random folds
folds <- sample(rep(1:10, length.out=nrow(D)))
# Perform 10-fold cross-validation
#apply: to avoid for loops,
#lapply: returns the output as a list, run on every element of data
#sapply: returns the output as a vector, run on every element of data

cv_errors <- sapply(1:10, function(i) { #means run the designated function with i = 1 to 5.
test_idx <- which(folds == i, arr.ind=TRUE)
train_set <- D[-test_idx, ]
test_set <- D[test_idx, ]
lda_model <- lda(Y ~ ., data=test_data_ten)
lda_pred <- predict(lda_model, test_data_ten)$class
mean(lda_pred != test_data_ten$Y) # Error for this fold
})
mean(cv_errors)
}
LDA10cv(train_data_ten)

[1] 0.06377678

In [ ]:

LDA10cv <- function(D) {
    # Making sure the outcome variable 'Y' is present in the dataset
    if (!"Y" %in% names(D)) {
        stop("The dataset must contain 'Y' as the target variable.")
    }

    # Create 10 random folds
    set.seed(123)  # Set a seed for reproducibility
    folds <- sample(rep(1:10, length.out = nrow(D)))

    # Perform 10-fold cross-validation
    cv_errors <- sapply(1:10, function(i) {
        test_idx <- which(folds == i)
        train_set <- D[-test_idx, ]
        test_set <- D[test_idx, ]

        lda_model <- lda(Y ~ ., data = train_set)
        lda_pred <- predict(lda_model, test_set)$class
        mean(lda_pred != test_set$Y)  # Error for this fold
    })

    mean(cv_errors)  # Return the average error across all folds
}

# Usage of the function with your training data
cv_error_lda <- LDA10cv(train_data)
print(cv_error_lda)


[1] 0.07680835


In [ ]:

# Function for 10-fold CV error for QDA
QDA10cv <- function(D) {


    # Set a random seed for reproducibility
    set.seed(123)
    folds <- sample(rep(1:10, length.out = nrow(D)))

    # Perform 10-fold cross-validation for QDA
    cv_errors <- sapply(1:10, function(i) {
        test_idx <- which(folds == i)
        train_set <- D[-test_idx, ]
        test_set <- D[test_idx, ]

        qda_model <- qda(Y ~ ., data = train_set)
        qda_pred <- predict(qda_model, test_set)$class
        mean(qda_pred != test_set$Y)  # Error for this fold
    })

    mean(cv_errors)  # Average error across all folds
}

# Usage of the function with your dataset
cv_error_qda <- QDA10cv(train_data_ten)  # Assuming full_data is your dataset
print(cv_error_qda)


ERROR: Error in qda.default(x, grouping, ...): rank deficiency in group 0
